# waves

In [2]:
#import math
from scipy import io
import scipy.signal
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack
import scipy.integrate
import pandas as pd
from IPython.display import display
import time
import matplotlib.colors as colors
import random

**main variables**
<br> f - main frequency (Hz)
<br> full_f - the entire time-frequency function (Hz) ("x" as x axis)
<br> frangemax - x axis length of Fourier-transform, note, that this does not affect Fourier calculation
<br> samples - sample points of the first graph
<br> s_rate - sample rate (Hz)
<br> amp - amplitude of signal
<br> randamp - amplitude of noise (random noise)
<br> <br> First, we create our signal and noise. In deafult: a sine wave, and a white noise.
<br> Or import one in.

In [3]:
#%%script false
f=40
#t=4
samples=500 #random.randint(400,50000)
s_rate=1000
t=samples/s_rate
randamp=0
nyquist=samples/2/t
frangemax=60#nyquist

xf=np.linspace(0.0, frangemax, int(frangemax*t+1))
x = np.linspace(0.0, t, samples)
full_f=np.linspace(f, f, samples) #(f-65/(x+1.1))  #   f/2+np.sin(2*np.pi*x)   f+x*0.01        
full_f_phase = np.zeros(samples)
for i in range (samples):
    full_f_phase[i]=np.sum(full_f[:i])/500
    
amp=1#0.4+x/t*0.6

s_det = amp*np.sin(full_f_phase*np.pi) #Changing freqency. old way: *(1+(x*10**2)**0.5)
s_noise = (np.random.rand(samples)-0.5)*randamp
s=(s_det+s_noise)/(0.5*randamp+1)
print (samples)
print (samples)
#f1=f; f2=f/np.sqrt(2); s=np.sin(f1*2*np.pi*x)+np.sin(f2*2*np.pi*x)

36531
36531


In [4]:
%%script false
sav1 = io.readsav('C:\\Users\\Marci\\Documents\\transientwaves\\imputdata\AUGD_34924_2208_2238_Loaded-with-MTR.sav', python_dict = True)
print(sav1)
s=np.array(sav1['data'][4])
x=np.array(sav1['timeax'])
#print(cpc_sav_dic1)
#print('\n\n',x)
#C:\\Users\\win10\\Downloads\\transient-waves\\inputdata\AUGD_34924_2208_2238_Loaded-with-MTR.sav
#C:\\Users\\Marci\\Documents\\transientwaves\\imputdata\AUGD_34924_2208_2238_Loaded-with-MTR.sav

Couldn't find program: 'false'


In [5]:
%%script false
f=0
s = s.astype(float)
x = x.astype(float)
samples=s.size
full_f = np.zeros(samples)
t=max(x)-min(x)
nyquist=samples/2/t
frangemax=100000#nyquist#
xf=np.linspace(0.0, frangemax, frangemax*t+1)
s_rate=x.size/t
print(s_rate)

Couldn't find program: 'false'


The widget below controls the wisible range of the chart under it. Note that this is only for visualisation, the program calculates with the whole 0 - t range. The zooming is exponential. Value 0 displays the whole range.

In [6]:
%%script false
usewidgets=1
import ipywidgets as widgets
view=widgets.FloatSlider()
if usewidgets==1:
    display(view)
else:
    view.value=0

Couldn't find program: 'false'


In [7]:
%%script false
plt.figure(figsize=(13,5))
plt.title('Input signal')
plt.plot(x,s)
#yaxismax=1/2**(view.value/12)
plt.axis([min(x),max(x)-(max(x)-min(x))*view.value/100.6,min(s),max(s)])
plt.ylabel('amplitude (V)')
plt.xlabel('time (s)')
plt.show()

plt.figure(figsize=(13,5))
plt.title("Input signal's frequency")
plt.plot(x,full_f)
#yaxismax=1/2**(view.value/12)
#plt.xlim(-0.01,t/(1.05**view.value))
plt.ylabel('Frequency (Hz)')
plt.xlabel('time (s)')
plt.grid(True)
plt.show()

print(full_f[-1]*t)

Couldn't find program: 'false'


## Fourier transform and examining it
<br>Now we take the Fourier-transform of the signal abowe, what has been already calculated, and make 3 plots:
 - One displaying the Real and imaginery parts of the vectors generated by Fourier at the corresponding frequencies
 - Next displaying their absolute values - we wil use these absolute values in the future
 - And one displaying the rotation of these vectors on the imaginary plane

In [8]:
%%script false
yf = scipy.fftpack.fft(s)
yfabs=np.abs(yf[:xf.size])
yfang=np.angle(yf[:xf.size])
yfimag=np.imag(yf[:xf.size])
yfreal=np.real(yf[:xf.size])
#yfabs=(np.abs(yf[:frangemax])*np.abs(yfimag)*np.abs(yfreal))

plt.figure(figsize=(13,5))
plt.plot(xf,yfimag,xf,yfreal)
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)')
plt.ylabel('Fourier amplitude')
plt.title('Fourier-transform real and imaginery parts')
plt.grid(True)
plt.legend(('Real', 'Imaginery'))
plt.show()

plt.figure(figsize=(13,5))
plt.plot(xf,yfabs,".-")
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)')
plt.ylabel('Fourier amplitude')
#plt.xscale('log')
plt.yscale('log')
plt.title('Fourier-transform absolute value')
plt.grid(True)
plt.show()

yfang=yfang/np.pi
plt.figure(figsize=(13,5))
plt.plot(xf,yfang,".-")
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)') 
plt.ylabel("Fourier phase (pi)")
#plt.xscale('log')
plt.title('Fourier-transform phase')
plt.grid(True)
plt.show()

#print(yfang[f*int(t)+1])
#print(yfang[f*int(t)])
#print(yfang[f*int(t)-1])

Couldn't find program: 'false'


Now, we will seek the peaks of the 2nd graph: we will search for the most likely original frequencies. We will find the 5 biggest values, and display their Fourier (absolute) values.

In [9]:
%%script false
#print (np.average(np.abs(yf)))
#print (yfabs)
yfsort=np.sort(yfabs,axis=None)
#print(yfsort)
fout=[0,0,0,0,0]
yfmaxsum=0
for i in range(5):
    #yfmaxsum=yfmaxsum+yfsort[-i-1]
    for j in range(xf.size):
        if yfsort[-i-1] == yfabs[j]:
            fout[i]=j/t
    #print (fout[i],"  ",yfabs[fout[i]])
#print (np.max(np.abs(yf)))

d = {'Frequency (Hz)': [fout[0],fout[1],fout[2],fout[3],fout[4]], 
     'Fourier amplitude': [yfabs[int(fout[0]*t)],yfabs[int(fout[1]*t)],yfabs[int(fout[2]*t)],yfabs[int(fout[3]*t)],yfabs[int(fout[4]*t)]]}
df = pd.DataFrame(data=d)
display (df)

deviation=np.std(yfabs)
certainity=yfabs[int(fout[0]*t)]/deviation

yfsum=np.sum(yfabs*yfabs)
certainity2=yfabs[int(fout[0]*t)]*yfabs[int(fout[0]*t)]/yfsum


if fout[0]==f:
    print ('We have succesfully found our original frequency:', f ,'Hz')
#    print ('\nConfidence using square sum:', 100*certainity2, '%')
#    print ('Confidence using standard deviation:', certainity)
else:
    print ("We haven't succesfully found our original frequency. It has drowned in noise, or the program is not good enough to detect it.")

Couldn't find program: 'false'


Now, we will F-transform the deterministic and stohastic component of our signal separatly, and compare them.

In [10]:
%%script false
yf_det=scipy.fftpack.fft(s_det)
yf_det_abs=np.abs(yf_det[:xf.size])
yf_noise=scipy.fftpack.fft(s_noise)
yf_noise_abs=np.abs(yf_noise[:xf.size])

plt.figure(figsize=(13,5))
plt.plot(xf,yf_det_abs)
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)')
plt.ylabel('Fourier value (m*s)')
#plt.xscale('log')
plt.title('Fourier-transform absolute value of deterministic signal')
plt.grid(True)
plt.show()

plt.figure(figsize=(13,5))
plt.plot(xf,yf_noise_abs)
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)')
plt.ylabel('Fourier value (m*s)')
plt.title('Fourier-transform absolute value non-deterministic signal')
plt.grid(True)
plt.show()

Couldn't find program: 'false'


## Short-time Fourier transform

In [11]:
t0=time.time()
hann_r = 120
d_win=30#random.randint(200,300)
hann_base = np.hanning(2*hann_r+1)

z_padded_s = np.linspace (0, 0, samples+2*hann_r)
z_0=z_padded_s
z_padded_s[hann_r:-hann_r] = s
x_hann=np.linspace(0.0, t+(2*hann_r/s_rate), samples+2*hann_r)

win = np.zeros((int(samples/d_win), samples+hann_r+hann_r))
for i in range(int(samples/d_win)):
    win [i][d_win*i:d_win*i+hann_r+hann_r+1]=hann_base

z_win = np.zeros((int(samples/d_win), samples+hann_r*2))
for i in range(int(samples/d_win)):
    z_win[i] = win[i] * z_padded_s

z_win_fft = np.zeros((int(samples/d_win), samples+hann_r*2))
for i in range(int(samples/d_win)):
    z_win_fft[i] = np.abs(scipy.fftpack.fft(z_win[i]))  
t1=time.time()

In [12]:
t2=time.time()
hann_big=np.zeros(samples)
for i in range(2*hann_r):
    hann_big[int(samples/2-hann_r+i)] = hann_base[i]

f_ax_stft, t_ax_stft, z_spy_stft = scipy.signal.stft(s,1000,window=hann_big, nperseg=samples, noverlap =samples-d_win)
t3=time.time()
print(z_win_fft.shape)
print(t1-t0)
print(t3-t2)

(1217, 36771)
4.55909538269043
1.3737118244171143


In [13]:
%%script false
print(len(f_ax_stft))
z_spy_stft_cutted = np.zeros((int(frangemax*t),int(samples/d_win)))
for i in range(int(samples/d_win)):
    for j in range(int(frangemax*t)):
        z_spy_stft_cutted [j][i] = np.abs(z_spy_stft [j][i])**2

z_win_fft_cutted = np.zeros((int(frangemax*max(x_hann)), int(samples/d_win)))
for i in range(int(samples/d_win)):
    for j in range(int(frangemax*max(x_hann))):
        z_win_fft_cutted [j][i] = z_win_fft [i][j]**2

Couldn't find program: 'false'


In [14]:
%%script false
logax = np.linspace(int(np.log10(z_win_fft_cutted.min())),int(np.log10(z_win_fft_cutted.max())),int(np.log10(z_win_fft_cutted.max()))-int(np.log10(z_win_fft_cutted.min()))+1)
logax = 10**logax
fig, ax = plt.subplots(figsize=(15,10))
#extent=[0,t-(2*hann_r/s_rate),0,frangemax/t],
#plt.figure(figsize=(15,10))
ax.plot(x,np.abs(full_f),color='w',linewidth=3,linestyle='dashed')
img = ax.imshow(z_win_fft_cutted,cmap='plasma', interpolation='none', aspect='auto',  extent=[min(x),max(x),frangemax,0])#,  norm=colors.LogNorm(vmin=z_win_fft_cutted.min(), vmax=z_win_fft_cutted.max()))
plt.title('My Short-time Fourier transform')
plt.ylabel('Frequency (Hz)')
plt.xlabel('time (s)')
#plt.xlim(2.234,2.238)
plt.subplots_adjust(right=0.88)
plt.gca().invert_yaxis()
cbar=plt.colorbar(img,cax=plt.axes([0.92, 0.2, 0.04, 0.6]))
cbar.set_label('Fourier value')
#cbar.set_ticks(logax)
plt.show()


logax = np.linspace(int(np.log10(z_spy_stft_cutted.min())),int(np.log10(z_spy_stft_cutted.max())),int(np.log10(z_spy_stft_cutted.max()))-int(np.log10(z_spy_stft_cutted.min()))+1)
logax = 10**logax
fig, ax = plt.subplots(figsize=(15,10))
#plt.figure(figsize=(15,10))
#plt.pcolormesh(f_ax, t_ax, np.abs(z_spy_stft), vmin=0, vmax=amp)
ax.plot(x,np.abs(full_f),color='w',linewidth=3,linestyle='dashed')
plt.title('Scipy short time Fourier transformation')
plt.ylabel('Frequency (Hz)')
plt.xlabel('time (s)')
plt.subplots_adjust(right=0.88)
img2 = ax.imshow(np.abs(z_spy_stft_cutted),cmap='plasma', interpolation='none', aspect='auto', extent=[min(x),max(x),frangemax,0])#, norm=colors.LogNorm(vmin=z_spy_stft_cutted.min(), vmax=z_spy_stft_cutted.max()))
plt.gca().invert_yaxis()
cbar=plt.colorbar(img2,cax=plt.axes([0.92, 0.2, 0.04, 0.6]),ticks=None)
cbar.set_label('Fourier value')
#cbar.set_ticks(logax)
plt.show()

Couldn't find program: 'false'


In [15]:
%%script false
plt.plot(hann_base,".-")
plt.title('Hann window')
plt.xlabel('sample no.')
plt.ylabel('Hann value')
plt.show()

plt.figure(figsize=(13,5))
plt.plot(x_hann,z_padded_s,".-")
plt.xlim((0,max(x_hann)))
plt.title('Padded signal')
plt.xlabel('time (s)')
plt.ylabel('Amplitude')
plt.show()

plt.figure(figsize=(10,5))
plt.imshow(win,aspect='auto')
plt.title('Sliding Hann window')
plt.xlabel('sample no')
plt.ylabel('sliding value')
#plt.subplots_adjust(right=0.95)
cax = plt.axes([0.8, 0.15, 0.03, 0.5])
plt.colorbar(cax=cax).set_label('Signal value')
plt.show()

plt.figure(figsize=(10,10))
plt.imshow(z_win,aspect='auto')
plt.title('Sliding Hann window + input signal')
plt.xlabel('sample no')
plt.ylabel('sliding value')
plt.subplots_adjust(right=0.88)
cax = plt.axes([0.92, 0.23, 0.04, 0.54])
plt.colorbar(cax=cax).set_label('Signal value')
plt.show()

Couldn't find program: 'false'


## Small observations
Now, we make histograms of the noise and the entire signal. The difference - if the input is a single frequency - only one point. In general: it is the signal. We will fit a half gaussian onto these, and see, how likely it is, that the difference (signal) is produced by the noise.

In [16]:
%%script false
hist_bins = np.linspace(0, 50000, 50)

"""plt.figure(figsize=(13,5))
plt.hist(yf_noise,hist_bins, density=True)
plt.title('Distribution of Fourier transformation (non-deterministic component)')
plt.ylabel('Frequency (%)')
plt.xlabel('Fourier amplitude')
plt.grid(True)
plt.show()"""

plt.figure(figsize=(13,5))
plt.hist(yfabs, hist_bins, density=True)
plt.title('Distribution of entire Fourier transformation')
plt.ylabel('Frequency (%)')
plt.xlabel('Fourier amplitude')
plt.yscale('log')
plt.grid(True)
plt.show()

Couldn't find program: 'false'


In [17]:
%%script false
plt.figure(figsize=(13,5))
plt.acorr(yfabs, maxlags=1400)
plt.axis([-1,1400,0,1])
#plt.xlim((0,frangemax))
plt.xlabel('$\Delta$ frequency (Hz)')
plt.ylabel('Autocorrelation value')
#plt.xscale('log')
plt.title('Autocorrelation of fourier value')
plt.grid(True)
plt.show()

def acf(x):
    result = np.correlate(x, x, mode='full')
    return result[int(result.size/2):]

s_acf=acf(s)
s_acf=s_acf/np.max(s_acf)
s_acf_fft_abs=abs(scipy.fftpack.fft(s))
s_acf_fft_ang=np.angle(scipy.fftpack.fft(s))/np.pi

plt.figure(figsize=(13,5))
plt.plot(x,s_acf)
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim(min(x),max(x))
plt.xlabel('$\Delta$t (s)')
plt.ylabel('Autocorrelation value')
#plt.xscale('log')
plt.title('Autocorrelation of input signal')
plt.grid(True)
#plt.show()

plt.figure(figsize=(13,5))
plt.plot(xf,s_acf_fft_abs[:xf.size])
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)')
plt.ylabel('Fourier value')
#plt.xscale('log')
plt.title("Autocorrelation's Fourier-transform's amplitude")
plt.grid(True)
#plt.show()

plt.figure(figsize=(13,5))
plt.plot(xf,s_acf_fft_ang[:xf.size])
#plt.axis([0,frangemax,0,1.5/rs])
plt.xlim((0,frangemax))
plt.xlabel('frequency (Hz)')
plt.ylabel('Fourier value')
#plt.xscale('log')
plt.title("Autocorrelation's Fourier-transform's phase")
plt.grid(True)
#plt.show()

Couldn't find program: 'false'


We can see, that the ratio of the values of the Fourier transform of the deterministic and non-deterministic components of the input signal is practically a constant, and it is influenced by the relative amplitude of these components in the original signal. Is this a coincidence? [drawing](https://i.redd.it/nnlrafulz5701.jpg)

In [18]:
%%script false
det_noise_fft_ratios = yfabs/np.abs(yf_noise[:xf.size]+yf_det[:xf.size])
print("Ratio of the ffts of deterministic and non-deterministic components:",'\naverage:',np.average(det_noise_fft_ratios))
print('Standard deviation:',np.std(det_noise_fft_ratios))
if 0<np.std(det_noise_fft_ratios)<0.001 :
    print("The deviation is practically zero.")

Couldn't find program: 'false'


In [24]:
for q in range(10000):
    f=40
    #t=4
    samples=random.randint(400,5000)
    s_rate=1000
    t=samples/s_rate
    randamp=0
    nyquist=samples/2/t
    frangemax=60#nyquist

    xf=np.linspace(0.0, frangemax, int(frangemax*t+1))
    x = np.linspace(0.0, t, samples)
    full_f=(f-65/(x+1.1)) #   f/2+np.sin(2*np.pi*x)   f+x*0.01    np.linspace(f, f, samples)    
    full_f_phase = np.zeros(samples)
    for i in range (samples):
        full_f_phase[i]=np.sum(full_f[:i])/500
    
    amp=0.4+x/t*0.6

    s_det = amp*np.sin(full_f_phase*np.pi) #Changing freqency. old way: *(1+(x*10**2)**0.5)
    s_noise = (np.random.rand(samples)-0.5)*randamp
    s=(s_det+s_noise)/(0.5*randamp+1)
       
    
    t0=time.time()
    hann_r = 120
    d_win=30#random.randint(200,300)
    hann_base = np.hanning(2*hann_r+1)

    z_padded_s = np.linspace (0, 0, samples+2*hann_r)
    z_0=z_padded_s
    z_padded_s[hann_r:-hann_r] = s
    x_hann=np.linspace(0.0, t+(2*hann_r/s_rate), samples+2*hann_r)

    win = np.zeros((int(samples/d_win), samples+hann_r+hann_r))
    for i in range(int(samples/d_win)):
        win [i][d_win*i:d_win*i+hann_r+hann_r+1]=hann_base

    z_win = np.zeros((int(samples/d_win), samples+hann_r*2))
    for i in range(int(samples/d_win)):
        z_win[i] = win[i] * z_padded_s

    z_win_fft = np.zeros((int(samples/d_win), samples+hann_r*2))
    for i in range(int(samples/d_win)):
        z_win_fft[i] = np.abs(scipy.fftpack.fft(z_win[i]))  
    t1=time.time()
    
    t2=time.time()
    hann_big=np.zeros(samples)
    for i in range(2*hann_r):
        hann_big[int(samples/2-hann_r+i)] = hann_base[i]

    f_ax_stft, t_ax_stft, z_spy_stft = scipy.signal.stft(s,1000,window=hann_big, nperseg=samples, noverlap =samples-d_win)
    t3=time.time()
    
    file = open("out.txt","a")
    file.write(str(t1-t0)+"\t"+str(t3-t2)+"\t"+str(samples)+"\n")
    file.close()
    print(str(q)+"\t"+ str(t1-t0)+"\t"+str(t3-t2)+"\t"+str(samples))

0	0.041979312896728516	0.011489391326904297	4440
1	0.04447054862976074	0.015625476837158203	4510
2	0.01562356948852539	0.0	1252
3	0.3123493194580078	0.031248807907104492	4261
4	0.05784487724304199	0.015623331069946289	3096
5	0.012174129486083984	0.0	1974
6	0.08899569511413574	0.018955707550048828	4258
7	0.030016660690307617	0.0019805431365966797	589
8	0.06499576568603516	0.018998384475708008	4286
9	0.0149993896484375	0.002023458480834961	1365
10	0.3175055980682373	0.004000663757324219	1823
11	0.08399629592895508	0.020995378494262695	4565
12	0.01704692840576172	0.003000020980834961	1938
13	0.5997035503387451	0.018954992294311523	3903
14	0.016998291015625	0.0030007362365722656	1748
15	0.29355597496032715	0.002946138381958008	1631
16	0.0647432804107666	0.0	1298
17	0.023783445358276367	0.0020012855529785156	1374
18	0.014003515243530273	0.0009944438934326172	1263
19	0.10076522827148438	0.022011756896972656	4954
20	0.35562896728515625	0.03795123100280762	4471
21	0.02199840545654297	0.0059993

201	0.43870997428894043	0.01562666893005371	3627
202	0.6793174743652344	0.0	2489
203	0.6637108325958252	0.0	2393
204	0.282559871673584	0.0	3196
205	0.015648365020751953	0.0	836
206	0.0312502384185791	0.015636682510375977	3200
207	0.015611886978149414	0.0	908
208	0.1006615161895752	0.0	3380
209	0.14061594009399414	0.0	1654
210	0.025858163833618164	0.0029692649841308594	1681
211	0.0	0.015645980834960938	1558
212	0.031249523162841797	0.0	3190
213	0.17131280899047852	0.0	2725
214	0.29128074645996094	0.007993698120117188	3822
215	0.17857050895690918	0.01227426528930664	4494
216	0.0	0.015619993209838867	905
217	3.3715665340423584	0.031252384185791016	4439
218	0.34459924697875977	0.031249284744262695	4597
219	0.1093747615814209	0.015623331069946289	4007
220	0.020943880081176758	0.004956960678100586	2360
221	0.0	0.0	1388
222	0.0	0.0	795
223	0.09374594688415527	0.0	2215
224	0.1586916446685791	0.0	1331
225	0.4103729724884033	0.0	2746
226	1.2282094955444336	0.015622138977050781	4234
227	0.0937480

KeyboardInterrupt: 

## Tasks
<br> Fix zooming
<br> Lower the redundancy (memory error)
<br> 
<br> debug
<br> Nyquist freq as frangemax - implemented
<br> Lookup pandas
<br> Load in datas
<br> Import and evaluate chirps
<br> Reconstruct wave frequencies and amplitudes
<br> Get the radial ?function of several wave types